In [2]:
# setup
from IPython.display import display, HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Shortest Paths


## Generalizing DFS and BFS search

We've now seen two types of search that are conceptually very similar.


`dfs_stack`:

- `node = frontier.pop()`
- Stack: Last in, first out


`bfs_serial`:

- `node = frontier.popleft()`
- Queue: First in, first out

Can we generalize these?

In [ ]:
from collections import deque

def generic_search(graph, source, get_next_node_fn):
    #Expects graph to be an adjacency list,
    #source is a vertex
    #get_next_node_fn is a function that determines how to get the next node from the frontier.
    #Returns a list of nodes in the connected component containing soruce.
    def generic_search_helper(visited, frontier):
        if len(frontier) == 0:
            return visited
        else:
            ## pick a node
            node = get_next_node_fn(frontier)
            print('visiting', node)
            visited.add(node)
            frontier.extend(filter(lambda n: n not in visited, graph[node]))
            return generic_search_helper(visited, frontier)
        
    frontier = deque()
    frontier.append(source)
    visited = set()
    return generic_search_helper(visited, frontier)

def bfs_fn(frontier):
    return frontier.popleft()

def dfs_fn(frontier):
    return frontier.pop()


In [ ]:

graph = {
            'A': {'B', 'C'},
            'B': {'A', 'D', 'E'},
            'C': {'A', 'F', 'G'},
            'D': {'B'},
            'E': {'B', 'H'},
            'F': {'C'},
            'G': {'C'},
            'H': {'E'}
        }


<center>
<img src="figures/dfs-graph.jpeg" width="40%"/>
</center>

In [ ]:

generic_search(graph, 'A', bfs_fn)

visiting A
visiting C
visiting B
visiting G
visiting F
visiting E
visiting D
visiting H


{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

<center>
<img src="figures/dfs-graph.jpeg" width="40%"/>
</center>

In [ ]:
generic_search(graph, 'A', dfs_fn)

visiting A
visiting B
visiting D
visiting E
visiting H
visiting C
visiting F
visiting G


{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

## Priority First Search

We could use the `get_next_node_fn` as a way to pick the **highest priority** node to visit at each iteration.

E.g., consider a Web crawler that prioritizes which pages to visit first
- more interesting pages
- pages that update more frequently


We'll see several algorithms that are instances of priority first search. 

## Weighted graphs

Up to now we have focused on unweighted graphs. 

For many problems, we need to associate real-valued **weights** to each edge.

E.g., a graph where nodes are cities and edges represent the distance between them. 

<center>
<img src="figures/weighted.png" width=60%/>
</center>

The **weight of a path** in the graph is the sum of the weights of the edges along that path.

The **shortest weighted path** (or just **shortest path**) between **s** and **e** is the one with minimal weight.



**What is the shortest path from s to e**?

Note that if the edges have negative weights, the shortest weighted path might not exist, since we may be able to loop forever within a negative cycle. 

Consider this example:

<img src="figures/inf.png" width=60%/>

What is the shortest path from $s$ to $e$?

> Infinite loop in the cycle $s \rightarrow a \rightarrow b \rightarrow a$, so shortest path is $-\infty$.

To avoid this, we assume that the weights are non-negative. We will see later that the Bellman-Ford algorithm can be applied to graphs where the edges have negative weights.

## Use BFS to solve shortest path problem?

We saw that we can use BFS to get the distance from the source to each node.

Can we use BFS to solve the shortest path problem for weighted graphs?

<img src="figures/bfs_fail.png" width=50%/>


BFS will:
- visit b
- visit a
- but, will not traverse the path from a to b, since it doesn't visit a node more than once

Thus, BFS will not discover that the shortest path from $s$ to $b$ is $s \rightarrow a \rightarrow b$.




How could we modify the graph so we can use BFS to find the shortest path? If the edge weights are natural numbers, we can expand them into paths whose lengths are the weights.

<img src="figures/bfs-fix.png" width=50%/>




What is work of creating this graph from the original?

BFS takes $O(|V|+|E|)$ time, but the number of edges in the new graph will equal to the sum of weights of all edges, say $W$. (Consider a simple chain.)

The input size is $O(|V| + |E| + \log W)$, because the input size of a number $W$ is the number of digits of $W$ which is $\log(W)$. The work is $O(W)$ in the worst case. Thus reducing shortest paths to BFS requires work that is exponential in input size!

## SSSP: Single-Source Shortest Path

Given a weighted graph $G=(V,E,w)$ and a source vertex $s$, the single-source shortest path (SSSP) problem is to find a shortest weighted path from $s$ to every other vertex in $V$.

What would be a brute-force solution to this problem?

Is there anything we could reuse to improve the brute-force solution?


Consider this figure:

<center>
<img src="figures/subpaths.png" width="60%"/>
</center>

Suppose that an oracle has told us the shortest paths from $s$ to all vertices except for the vertex $v$, shown in red squares. How can we find the shortest path to $v$?



Let $\delta_G(i,j)$ be the weight of shortest path from $i$ to $j$ in graph $G$. Then:

$$
\begin{align}
\delta_G(s,v) = \min(&\delta_G(s,a)+3,\\
&\delta_G(s,b)+6,\\
&\delta_G(s,c)+5 )
\end{align}
$$



### sub-paths property
> any sub-path of a shortest path is itself a shortest path. 

The sub-paths property makes it possible to construct shortest paths from smaller shortest paths. 

> If a shortest path from New Orleans to Boston goes through Atlanta, then that path is built from shortest path from New Orleans to Atlanta, and from Atlanta to Boston.

## Dijkstra's property

Suppose we have a set of vertices $X$ whose shortest paths are known. Can we efficiently determine the shortest path to any of the vertices in its frontier? 

For any partitioning of vertices $V$ into $V=X\cup Y$ where

- $X$
    - The set of vertices that we know the shortest paths to
- $Y = V \setminus X$
    - everything else

Given a source vertex $s \in X$ and a vertex $v\in Y$, by the sub-paths property:

$$\delta_G(s,v) = \min_{v^\prime \in V(G)} (\delta_G(s,v^\prime) + w(v^\prime, v)) \geq \min_{x \in X} (\delta_G(s,x) + w(x,v))$$

>The rightmost expression is the length of the shortest path to $v$, assuming the rest of the path is in $X$.


Now, assume that $y$ is the closest vertex in $Y$ to $s$. By this assumption, the fact that $w(v^\prime,v)\geq 0$, and the sub-paths property, we must have $v^\prime\in X$ when the minimum is achieved and the inequality above becomes an equality.

Thus, we have found $\delta_G(s,v)$ and can add $v$ to $X$. We can use this reasoning to iteratively add vertices to $X$.


<center>
<img src="figures/dijkstra_example.jpg" width=50%/>
</center>

Which vertex will be added to $X$ next?



By going through vertex $d$, a path exists to get to $u$ with a cost of $9$.

$~~~~~\delta_G(s,u) = \delta_G(s,d) + w(d,u) = 5 + 4 = 9$

The shortest path to vertex $v$ is through $b$ with a cost of 10.

$~~~~~\delta_G(s,v) = \delta_G(s,b) + w(b,v) = 4 + 6 = 10$




### Full details

<center>
<img src="figures/dijkstra_example.jpg" width=45%/>
</center>

$$
\begin{align}
\delta_G(s,v) = \min(&\delta_G(s,a) + w(a,v) = 8 + 3 = 11,\\
&\pmb{\delta_G(s,b) + w(b,v) = 4 + 6 = 10}, \\
&\delta_G(s,c) + w(c,v) = 7 + 5 = 12) \\
\end{align}
$$

$$
\begin{align}
\delta_G(s,u) = 
\min(&\delta_G(s,c) + w(c,u) = 7 + 3 = 10, \\
&\pmb{\delta_G(s,d) + w(d,u) = 5 + 4 = 9}
)
\end{align}
$$

$\texttt{Vertex } \pmb{u}$ will be added to $X$ next.

## Iteratively expanding our known vertices

The sub-paths property suggests that we can continue to expand the set of our "known" vertices by identifing and adding the the frontier vertex at the shortest distance from $s$.



This will ensure that we visit nodes in an order to identify actual shortest paths.

<center>
<img src="figures/bfs_fail.png" width=50%/>
</center>

By visiting nodes in order of increasing distance from the source, we will find the shortest path from $a \rightarrow c$.



In the following figure, assume we know the shortest paths to $\{a,b,c,e\}$. We can then use these to determine which of $u$ or $v$ is closer to $s$.

<center>
<img src="figures/distance.png" width=50%/>
</center>


The idea of Dijkstra's algorithm is:
- Maintain a visited set of vertices whose distances have already been computed correctly.
- Calculate distances to each node in the frontier.
- Extend the frontier by visiting the closest vertex.


## Dijkstra's Algorithm

The final algorithm can be viewed as an instance of **priority-first search**, using the path length as the priority criterion.

1. Initialize frontier to $(s, 0)$
2. While frontier not empty:
  - pop from the frontier the node $\pmb{v}$ with minimum distance $\pmb{d}$ from the source. (You can break ties arbitrarily.)
  - set $\texttt{result}(v) = d$
    - This is the weight of the shortest path from $\pmb{s}$ to $\pmb{v}$.
  - For each neighbor $\pmb{n}$ of $\pmb{v}$ with edge weight $\pmb{w}$, add $\pmb{n}$ to frontier with distance $\pmb{d} + \pmb{w}$. Note $\pmb{n}$ may already be in the frontier, so it might be in the frontier multiple times.
3. return $\texttt{result}$

## Example by hand

<center>
    <img src="figures/dijkstra-0.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-1.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-2.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-3.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-4.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-5.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-6.jpg" width=50%/>
</center>

<center>
    <img src="figures/dijkstra-7.jpg" width=50%/>
</center>

## Implementing priority queues via heaps.

A heap is a binary tree where parent nodes are smaller than the child nodes. This means that the smallest node is the root, and we can sort the nodes by continually extracting the root. Heaps come with the operations push and pop, which add and remove nodes from the heap respectively. By continually popping, we can sort the nodes in the heap.

In [3]:
# Heaps in Python
from heapq import heappush, heappop 
  
# Creating empty heap 
heap = [] 
  
# Adding items to the heap using heappush function 
heappush(heap, (10, 'a')) 
heappush(heap, (30, 'b')) 
heappush(heap, (20, 'c')) #We put b and c into the heap out of order.
heappush(heap, (400, 'd')) 
print("Head value of heap : "+str(heappop(heap)))
print("Head value of heap : "+str(heappop(heap)))
print("Head value of heap : "+str(heappop(heap))) #b and c come out of the heap in order.
print("Head value of heap : "+str(heappop(heap)))

Head value of heap : (10, 'a')
Head value of heap : (20, 'c')
Head value of heap : (30, 'b')
Head value of heap : (400, 'd')


In [ ]:
def dijkstra(graph, source):
    def dijkstra_helper(visited, frontier):
        if len(frontier) == 0:
            return visited
        else:
            # Pick next closest node from heap
            distance, node = heappop(frontier)
            print('visiting', node)
            if node in visited:
                # Already visited, so ignore this longer path
                return dijkstra_helper(visited, frontier)
            else:
                # We now know the shortest path from source to node.
                # insert into visited dict.
                visited[node] = distance
                print('...distance=', distance)
                # Visit each neighbor of node and insert into heap.
                # We may add same node more than once, heap
                # will keep shortest distance prioritized.
                for neighbor, weight in graph[node]:
                    heappush(frontier, (distance + weight, neighbor))                
                return dijkstra_helper(visited, frontier)
        
    frontier = []
    heappush(frontier, (0, source))
    visited = dict()  # store the final shortest paths for each node.
    return dijkstra_helper(visited, frontier)


<center>
    <img src="figures/dijkstra-0.jpg" width=50%/>
</center>

In [ ]:

graph = {
            's': {('a', 1), ('c', 5)},
            'a': {('b', 2)},
            'b': {('c', 1), ('d', 5)}, 
            'c': {('a', 2), ('d', 3)},
            'd': {},
            'e': {('d', 0)}
        }
        


<center>
    <img src="figures/dijkstra-7.jpg" width=50%/>
</center>

In [ ]:
dijkstra(graph, 's')

visiting s
...distance= 0
visiting a
...distance= 1
visiting b
...distance= 3
visiting c
...distance= 4
visiting c
visiting a
visiting d
...distance= 7
visiting d


{'s': 0, 'a': 1, 'b': 3, 'c': 4, 'd': 7}

### Correctness of Dijkstra's Algorithm

The algorithm maintains an invariant that each visited element $x \in X$ contains the shortest path from $s$ to $x$.
- That is, `visited[x]` $=\delta_G(s,x)$


- We know this is true after visiting the source, since $\delta_G(s,x)=$ `visited[x]` $=0$
- Dijkstra's property ensures that each element we remove from the heap also maintains this property
- Like most invariants, this can be proven through induction.

## Work of Dijkstra's Algorithm

The two key lines are:

```python
distance, node = heappop(frontier)
```

and


```python
for neighbor, weight in graph[node]:
    heappush(frontier, (distance + weight, neighbor))
```    

<br>

What is work and span of `heappop` and `heappush`?

The worst case is $O(\log n)$ work and span for both push and pop, where $n$ is the number of elements in the heap. Heaps maintain a balanced structure, so there is no linear-time worst case, unlike binary search trees.

<br>

How many times will we call these functions?



Once per edge, since a node may be added to the heap multiple times for each edge.


<br>

Total work?


$O(|E| \log |E|)$




Note that we assume constant time `dict` operations:
- `visited[node] = distance`
- `for neighbor, weight in graph[node]:`
- These result in an additional $O(|V| + |E|)$ work, but are dominated by the above.

Because this is a serial algorithm, the span is also $O(|E| \log |E|)$

### Potential Optimization



<center>
    <img src="figures/dijkstra-2.jpg" width=40%/>
</center>

<center>
    <img src="figures/dijkstra-3.jpg" width=40%/>
</center>

When we visit vertex $b$, we add $c$ to the queue a second time.



Alternatively, we could update the existing value and reheap up or down as necessary.

This would ensure that our heap is never larger than $|V|$, giving us $O(\log |V|)$ `heappush` and `heappop` operations rather than $O(\log |E|)$ ones.

With this optimization, we can get a runtime for Dijkstra's of:

$O(|E| \log |V|)$

> What is the shortest way to travel from Rotterdam to Groningen, in general: from given city to given city. It is the algorithm for the shortest path, which I designed in about twenty minutes. One morning I was shopping in Amsterdam with my young fiancée, and tired, we sat down on the café terrace to drink a cup of coffee and I was just thinking about whether I could do this, and I then designed the algorithm for the shortest path. 
> 
> As I said, it was a twenty-minute invention. In fact, it was published in '59, three years later. The publication is still readable, it is, in fact, quite nice. One of the reasons that it is so nice was that I designed it without pencil and paper. I learned later that one of the advantages of designing without pencil and paper is that you are almost forced to avoid all avoidable complexities. Eventually, that algorithm became to my great amazement, one of the cornerstones of my fame.

— Edsger Dijkstra, in an interview with Philip L. Frana, Communications of the ACM, 2001